***Participantes (RM - NOME):***<br>
339624 - Camila<br>
339656 - Cleiton<br>
340214 - Henrique<br>
339708 - Roberto<br>
340192 - Sergio<br>

## **Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

# Bibliotecas utilizadas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import wordcloud as wd
import re
from collections import Counter

import spacy

import nltk
from nltk.stem.rslp import RSLPStemmer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle

import tensorflow as tf
import keras
from keras.backend import clear_session
from keras.callbacks import ModelCheckpoint   
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, SpatialDropout1D, LSTM
from keras.layers import Dropout, Flatten, Conv1D, Embedding
from keras.layers import GlobalMaxPooling1D
from keras.utils.vis_utils import plot_model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.convolutional import MaxPooling1D
import pydot


### carregando os modelos de lemmatização e stemização

In [ ]:
sb.set_theme(context='notebook', style='whitegrid')

nlp = spacy.load('pt_core_news_md')
rslp = RSLPStemmer()

### checando se existe GPU disponível

In [ ]:
print(tf.__version__)
#print(keras.__version__)
print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Funções Auxiliares

In [ ]:
# função de Stemização completa do documento
def stemmer_text(frase):
  tokens = [rslp.stem(w) for w in frase.split()]
  return " ".join(tokens)

# função de lematização completa do documento
def lemmatizer_text(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ for w in doc]
  return " ".join(tokens)

# função de lematização somente para os verbos do documento
def lemmatizer_verbs(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ if w.pos_ == 'VERB' else w.text for w in doc]
  return " ".join(tokens)

# função de lematização apenas ADV e ADJ
def retirar_stop_words(frase):
  palavras = frase.split()
  palavras = [p.lower() for p in palavras if p not in stops]
  return " ".join(palavras) 

# função para retiral o plural das palavras em português
def plural_singular(palavra):
  r2 = r"ses$|zes$|res$"
  r3 = r"ões$|ães$"
  r4 = r"ais$|éis$|óis$|uis$"
  r5 = r"is$"
  r6 = r"eis$"
  if palavra.endswith('s'):
    if re.findall(r2, palavra):
      return palavra[:-2]
    if re.findall(r3, palavra):
      return palavra[:-3] + "ão"
    if re.findall(r4, palavra):
      return palavra[:-2] + "l"
    if re.findall(r5, palavra):
      return palavra[:-1] + "l"
    if re.findall(r6, palavra):
      return palavra[:-1] + "il"
    if palavra.endswith('ns'):
      return palavra[:-2] + "m"
    return palavra[:-1]
  return palavra  

# função para executar o pré processamento
def pre(frase):
  regex = r"[`,.?:;!&\"]"
  palavras = frase.split()
  palavras = [p.lower() for p in palavras if p not in stops]
  palavras = [re.sub(regex, "", p) for p in palavras]
  palavras = [p for p in palavras if len(p) >= 3]
  palavras = [p for p in palavras if not p.isnumeric()]
  palavras = [plural_singular(p) for p in palavras]

  return " ".join(palavras)

# função para criar as bases de treino e teste vetorizadas, com Unigramas, Bigramas e Trigramas
def criar_vetores(base_treino, base_teste, coluna):
    nomes = ['CV ngram(1,1)', 'CV ngram(1,2)', 'CV ngram(1,3)', 'CV ngram(2,2)', 'CV ngram(2,3)', 'CV ngram(3,3)',
                'TF ngram(1,1)', 'TF ngram(1,2)', 'TF ngram(1,3)', 'TF ngram(2,2)', 'TF ngram(2,3)', 'TF ngram(3,3)',]
    vetores = []
    idx = 0

    for x in range(1,4):
        for y in range(1,4): 
            if (y < x):
                continue
            vect = CountVectorizer(ngram_range=(x,y), stop_words = stops, min_df=10)
            vect.fit(df_treino[coluna])
            vect_treino = vect.transform(df_treino[coluna])
            vect_teste = vect.transform(df_teste[coluna])
            vetores.append([nomes[idx], vect_treino, vect_teste])
            idx += 1

    for x in range(1,4):
        for y in range(1,4): 
            if (y < x):
                continue
            vect = TfidfVectorizer(ngram_range=(x,y), stop_words = stops, min_df=10)
            vect.fit(df_treino['texto'])
            vect_treino = vect.transform(df_treino[coluna])
            vect_teste = vect.transform(df_teste[coluna])
            vetores.append([nomes[idx], vect_treino, vect_teste])
            idx += 1

    return vetores

def testar_modelo(modelo, base_treino, base_teste, vetores, coluna):
    f1score = 0
    nome = ""
    for x in vetores:
        modelo.fit(x[1], base_treino[coluna])
        predito = modelo.predict(x[2])
        f1 = f1_score(base_teste[coluna], predito, average='weighted')
        print(x[0], f1)
        if (f1score < f1):
            f1score = f1
            nome = x[0]

    return (nome, f1score)    

# Carregando os Dados

In [ ]:
df = pd.read_csv('./data/reviews-pt-br.csv', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
df.head(10)

## Conferindo se temos dados nulos ou duplicados

In [ ]:
df.isna().sum()

In [ ]:
df[df.duplicated()].count()

# Distribuição das respostas

In [ ]:
df.groupby('sentimento').count()

In [ ]:
# em percentual
round(df.groupby('sentimento').count().texto / df.shape[0] * 100, 2)

In [ ]:
sb.countplot(x=df.sentimento)

# Criando o conjunto de stopwords (NLTK + SPACY)

In [ ]:
stopwords_nltk = nltk.corpus.stopwords.words('portuguese')
stopwords_spacy = nlp.Defaults.stop_words
stops = list(set(stopwords_spacy).union(stopwords_nltk))
print(sorted(stops))

# Analisando o conjunto total das palavras

In [ ]:
# exemplo de críticas no nosso dataframe
print(df.texto[0])
print("-" * 40)
print(df.texto[1536])
print("-" * 40)
print(df.texto[8192])

In [ ]:
# criando um texto completo com todas as críticas
texto_completo = " ".join([texto for texto in df.texto])
texto_completo_pos = " ".join([texto for texto in df[df.sentimento == 'pos'].texto])
texto_completo_neg = " ".join([texto for texto in df[df.sentimento == 'neg'].texto])

In [ ]:
print("QTD de todas as palvaras", len(texto_completo.split()))
print("QTD de todas as palvaras das críticas positivas", len(texto_completo_pos.split()))
print("QTD de todas as palvaras das críticas negativas", len(texto_completo_neg.split()))

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_pos)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras POSITIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_neg)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras NEGATIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

## Criando uma função de pré processamento

In [ ]:
palavras = texto_completo.split()
len(palavras)

In [ ]:
# retirando as stopwords das palavras
palavras = [p.lower() for p in palavras if p not in stops]
len(palavras)

In [ ]:
# retirando pontuações das palavras
regex = r"[`,.?:;!&\"]"
palavras = [re.sub(regex, "", p) for p in palavras]
len(palavras)

In [ ]:
#retirando palavras menores que 3 caracteres
palavras = [p for p in palavras if len(p) >= 3]
len(palavras)

In [ ]:
# retirando os números das palavras
palavras = [p for p in palavras if not p.isnumeric()]
len(palavras)

In [ ]:
#convertendo as palavras para o singular
palavras = [plural_singular(p) for p in palavras]
len(palavras)

## Criamos uma função chamada "pre" que iremos usar em nossas análises dos modelos

# Vamos analisar quais palavras são comuns as críticas negativas e positivas

In [ ]:
# criando uma wordcloud das palavras filtradas
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False).fit_words(dict(ctp))

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados após o Pré Pocessamento", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
todas_palavras_pos = texto_completo_pos.split()
todas_palavras_pos = [pre(w) for w in todas_palavras_pos]
todas_palavras_pos = [w for w in todas_palavras_pos if w != '']
ctpp = Counter()
ctpp.update(todas_palavras_pos)
ctpp.most_common(50)

In [ ]:
todas_palavras_neg = texto_completo_neg.split()
todas_palavras_neg = [pre(w) for w in todas_palavras_neg]
todas_palavras_neg = [w for w in todas_palavras_neg if w != '']
ctpn = Counter()
ctpn.update(todas_palavras_neg)
ctpn.most_common(50)

In [ ]:
%%time
p = ctpp.most_common()
n = ctpn.most_common()
r = [[x for x in p if x[0] == y[0]] for y in n]
r = [x for x in r if len(x) != 0]
len(r)

In [ ]:
palavras_comuns = [w[0][0] for w in r]
palavras_comuns[:100]

## Criando as colunas tratadas

In [ ]:
%%time
#df['stem'] = df.texto.apply(stemmer_text)

In [ ]:
%time
#df['lemm'] = df.texto.apply(lemmatizer_text)

In [ ]:
%%time
#df['verb'] = df.texto.apply(lemmatizer_verbs)

In [ ]:
%%time
#df['pre'] = df.texto.apply(pre)

In [ ]:
%%time
#df['adj'] = df.texto.apply(lemmatizer_adv_adj)

In [ ]:
#df.to_csv("./data/criticas.csv")
df = pd.read_csv('../criticas.csv')
df = df.drop(['Unnamed: 0', 'codigo'], axis=1)

In [ ]:
df.texto[2]

In [ ]:
df.lemm[2]

In [ ]:
df.verb[2]

In [ ]:
df.stem[2]

In [ ]:
df.pre[2]

In [ ]:
df.head(10)

# Dividindo a base em treino e teste

In [ ]:
# dividindo com 20% para o treino e random state = 42
df_treino, df_teste = train_test_split(
      df, 
      test_size = 0.2, 
      random_state = 42
  )

In [ ]:
# distribuição das respostas do treino em %
round(df_treino.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

In [ ]:
# distribuição das respostas do teste em %
round(df_teste.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

# Testando diversos modelos com a base total de palavras sem stopwords

## Criando os vetores com CountVectorizer e TfidfVectorizer

In [ ]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'texto')

## Testando diversos modelos com os vetores criados

In [ ]:
%%time
# modelo Árvore de Decisão
modelo = DecisionTreeClassifier(random_state=42, max_depth=50)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Árvore de Decisão - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Random Forest
modelo = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Random Forest - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo AdaBoost
modelo = AdaBoostClassifier(n_estimators=100, random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"AdaBoost - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Bagging Classifier
modelo = BaggingClassifier(n_estimators=20, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Bagging - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Extra Trees Regressor
modelo = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Extra Trees Regressor - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Gradient Boosting
modelo = GradientBoostingClassifier(n_estimators=200, random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Gradient Boosting - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Passive Agressive
modelo = PassiveAggressiveClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Passive Agressive - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Ridge
modelo = RidgeClassifier(solver='sparse_cg', random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Ridge - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo SGD
modelo = SGDClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SGD - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Complement
modelo = ComplementNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Complement - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

# Testando os modelos com a base stematizada sem stopwords

In [ ]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'stem')

## apenas modelos com mais de 85% no passo anterior e tempo inferior a 5 minutos

In [ ]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Passive Agressive
modelo = PassiveAggressiveClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Passive Agressive - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Ridge
modelo = RidgeClassifier(solver='sparse_cg', random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Ridge - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo SGD
modelo = SGDClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SGD - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Complement
modelo = ComplementNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Complement - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

# Testando os modelos com a base lematizada sem stopwords

In [ ]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'lemm')

## continuamos com os mesmos modelos da Stematização

In [ ]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Passive Agressive
modelo = PassiveAggressiveClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Passive Agressive - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Ridge
modelo = RidgeClassifier(solver='sparse_cg', random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Ridge - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo SGD
modelo = SGDClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SGD - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Complement
modelo = ComplementNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Complement - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

# Testando os modelos com a base Pré Processada

In [ ]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'pre')

## Vamos testar apenas os 5 melhores modelos até agora

In [ ]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Extra Trees Regressor
modelo = ExtraTreesClassifier(n_estimators=10, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Extra Trees Regressor - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

# Testando os modelos com a base Lemmatizando apenas os verbos

In [ ]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'verb')

In [ ]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

In [ ]:
%%time
# modelo Extra Trees Regressor
modelo = ExtraTreesClassifier(n_estimators=20, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Extra Trees Regressor - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

# TESTE FINAL DOS DOIS MELHORES MODELOS

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,3), stop_words=stops)
vect.fit(df_treino.texto)
text_vect_treino = vect.transform(df_treino.texto)
text_vect_teste = vect.transform(df_teste.texto)

In [ ]:
%%time
# treinamento do modelo
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
modelo.fit(text_vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste (textos vetorizados)
predicao = model.predict(text_vect_teste)

# mensuração do resultado por F1 Score
f1score = f1_score(df_teste.sentimento, predicao, average='weighted')

print()
print(f"Regressão Logística - TF (1,3) - F1 Score: {round(f1score[1]*100,2)}%")

# Testando um modelo de MLP com a base pré processada

In [ ]:
import gensim
from gensim.models import KeyedVectors

#model_cbow = KeyedVectors.load_word2vec_format('../codigo/cbow_s300.txt')

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding='UTF-8') as f:
        primeira = f.readline()
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
# função para plotar os gráficos de acurácia e perda

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
# base para o treino de MLP
df_mlp = df[['pre']].copy()
df_mlp['target'] = [0 if x == 'neg' else 1 for x in df.sentimento]
df_mlp.head()

In [ ]:
# dividindo com 20% para o treino e random state = 42
df_treino_mlp, df_teste_mlp = train_test_split(
      df_mlp, 
      test_size = 0.2, 
      random_state = 42
  )
df_mlp.info()  

## Criação dos vetores para treinar o modelo

In [ ]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(df_treino_mlp.pre)

In [ ]:
X_treino = tokenizer.texts_to_sequences(df_treino_mlp.pre)
X_teste = tokenizer.texts_to_sequences(df_teste_mlp.pre)

In [ ]:
# vocabulário do treino
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
print(df_treino_mlp.pre[2])
print(X_treino[2])

In [ ]:
max(len(x) for x in X_treino)

In [ ]:
# normalizando o tamanho dos vetores das frases para iniciar o treino

maxlen = 430 # texto - 930

X_treino = pad_sequences(X_treino, padding='post', maxlen=maxlen)
X_teste = pad_sequences(X_teste, padding='post', maxlen=maxlen)

In [ ]:
X_treino.shape

In [ ]:
y_treino = df_treino_mlp.target.values.reshape(-1,1)
y_teste = df_teste_mlp.target.values.reshape(-1,1)
y_treino.shape

In [ ]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))


    for word in word_index:
        if model_cbow.has_index_for(word):
            idx = word_index[word]
            vector = model_cbow.get_vector(word)
            embedding_matrix[idx] = np.array(
                vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
# Não utilizei pois não melhorou o modelo e demorava demais para processar
# embedding_matrix = create_embedding_matrix(tokenizer.word_index, 256)

In [ ]:
#Modelo da Rede MLP

model = Sequential()
#model.add(Embedding(input_dim=vocab_size, output_dim=256, weights=[embedding_matrix],  input_length=maxlen))
model.add(Embedding(input_dim=vocab_size, output_dim=256,  input_length=maxlen))
model.add(Dropout(0.3))
#model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
plot_model(model)

In [ ]:
%%time
clear_session()
checkpointer = ModelCheckpoint(filepath='./modelo_mlp.hdf5', verbose=1,  save_best_only=True, monitor='val_accuracy')

#history  = model.fit(X_treino, y_treino, epochs=15, validation_data=(X_teste, y_teste), callbacks=[checkpointer], batch_size=1024)
history  = model.fit(X_treino, y_treino, epochs=20, validation_split=0.2, callbacks=[checkpointer], batch_size=1024)

In [ ]:
plot_history(history)

In [ ]:
plot_history(history)

In [ ]:
model = load_model("./modelo_mlp.hdf5")

In [ ]:
scores = model.evaluate(X_treino, y_treino)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
scores = model.evaluate(X_teste, y_teste)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.